In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import pandas as pd
import argparse
import string
import pandas as pd
import json
from utils.utils import get_logs
import time
from tqdm import tqdm
import copy
import re

In [2]:
# unused
def get_total_answer_num():
    cnt = driver.find_element(value="#info\.search\.place\.cnt",by='css selector').text
    return cnt

In [3]:
def get_data_from_page():
    c = 1
    while True:
        try:
            name = driver.find_element(value="#info\.search\.place\.list > li:nth-child(%d) > div.head_item.clickArea > strong > a.link_name"%c,by='css selector').text
            address = driver.find_element(value="#info\.search\.place\.list > li:nth-child(%d) > div.info_item > div.addr > p:nth-child(1)"%c,by='css selector').text
            score = driver.find_element(value="#info\.search\.place\.list > li:nth-child(%d) > div.rating.clickArea > span.score > em"%c,by='css selector').text
            dic = dict(name=name, address = address, score = score)
            if dic in answer_list:
                return True
            answer_list.append(dic)
            c+=1
        except:
            return False

In [4]:
MANGO_URL = 'https://www.mangoplate.com/'
KAKAO_URL = 'https://map.kakao.com/'
GOOGLE_URL = 'https://www.google.com/maps'
# OK - 기존과 바뀜 - 함수 통일화
ALPHABET = list(string.ascii_uppercase)

logger_1, logger_2 = get_logs()
parser = argparse.ArgumentParser(description='parse')
# argument는 원하는 만큼 추가한다.
parser.add_argument('--output_dir', type=str, default = './output/seoul/kakao')
parser.add_argument('--data_dir', type=str, default = './data/%s.json')
parser.add_argument('--where', type=str, choices=['mango','google','kakao'], default='kakao')
parser.add_argument('--wait_second', type=int, default = 2)
args, _ = parser.parse_known_args()

if args.where == 'mango':
    default_url = MANGO_URL
    
elif args.where == 'kakao':
    default_url = KAKAO_URL
    
elif args.where == 'google':
    default_url = GOOGLE_URL    

In [5]:
options = webdriver.chrome.options.Options()
#options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
#options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
#options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
driver = webdriver.Chrome('./chromedriver', options=options)

driver.get(default_url)

/tmp/ipykernel_18669/2884935049.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver', options=options)


In [ ]:
# data load
gus = [i.strip() for i in open('./data/서울시_자치구.txt','r',encoding='utf-8').readlines()]
for gu in gus:
    data = json.load(open(args.data_dir%gu))
    unique_data = {}
    for i,j in data.items():
        if i.split()[-1].endswith('점'):
            query = ' '.join(i.split()[:-1])
        else:
            query = i
        unique_data[query]=j
    print(gu)
    now = time.time()
    ####################################################################################
    for query, value in tqdm(unique_data.items()):
        driver.get(default_url)
        driver.implicitly_wait(args.wait_second)
        answer_list = []
        value['%s_score'%args.where]=[]
        driver.find_element(value="#search\.keyword\.query",  by = 'css selector').send_keys('서울 '+query)
        driver.find_element(value="#search\.keyword\.query",  by = 'css selector').send_keys(Keys.RETURN)
        check = False
        try:
            # 1 page 이상인 경우
            driver.find_element(value="#info\.search\.place\.more",by='css selector').click()
            time.sleep(args.wait_second)
            while True:
                page_list = driver.find_element(value="#info\.search\.page > div",by='css selector').text.split()[1:-1]
                for p in page_list:
                    change_p = int(p)%5 
                    if change_p == 0:
                        change_p = 5
                    driver.find_element(value="#info\.search\.page\.no%s"%change_p,by='css selector').click()
                    time.sleep(args.wait_second)
                    check = get_data_from_page()
                    if check:
                        break
                if check:
                    break
                driver.find_element(value="#info\.search\.page\.next",by='css selector').click()
                driver.implicitly_wait(args.wait_second)

        except:
            get_data_from_page()
        value['%s_score'%args.where]=copy.deepcopy(answer_list)
        logger_1.info(query)
        logger_1.info(value)
        continue
    ####################################################################################
    print(time.time()-now)
    json.dump(unique_data,open(os.path.join(args.ouput_dir,'%s.json'%gu),'w'))

강남구


  0%|▏                                      | 10/2741 [01:03<3:57:20,  5.21s/it]